# Neural Morphological Tagger / Disambiguator

EstNLTK contains neural morphological tagger / disambiguator that was introduced by [Tkachenko and Sirts (2018)](https://arxiv.org/pdf/1810.06908.pdf). 
It takes Vabamorf's analyses as an input to predict morphological tags (`partofspeech` and `form`) with better accuracy than Vabamorf. 
The tagger also employs a morphological tagset that extends Vabamorf's tags towards [UD](https://universaldependencies.org/guidelines.html)'s morphological features. 
As for the limitation, the tagger does not resolve lemma ambiguities.

## Running as a web tagger

Easiest way to use neural morphological tagger is via EstNLTK's web service:

In [1]:
from estnltk import Text
from estnltk.web_taggers import NeuralMorphDisambWebTagger

neural_morph_tagger = NeuralMorphDisambWebTagger(url='http://127.0.0.1:5000/estnltk/tagger/neural_morph_disamb')
neural_morph_tagger

NeuralMorphDisambWebTagger(url=http://127.0.0.1:5000/estnltk/tagger/neural_morph_disamb, batch_layer=words, batch_layer_max_size=125, batch_enveloping_layer=sentences)

In [2]:
# Create input text
text=Text('Kiirelt võetud pangalaen on kärmelt kulunud.').tag_layer('morph_analysis')
# Add neural morph layer
neural_morph_tagger.tag(text)
text['neural_morph_disamb']

Layer(name='neural_morph_disamb', attributes=('morphtag', 'pos', 'form'), spans=SL[Span('Kiirelt', [{'morphtag': 'POS=S|NOUN_TYPE=com|NUMBER=sg|CASE=abl', 'pos': 'S', 'form': 'sg abl'}]),
Span('võetud', [{'morphtag': 'POS=A|DEGREE=pos', 'pos': 'A', 'form': ''}]),
Span('pangalaen', [{'morphtag': 'POS=S|NOUN_TYPE=com|NUMBER=sg|CASE=nom', 'pos': 'S', 'form': 'sg n'}]),
Span('on', [{'morphtag': 'POS=V|VERB_TYPE=aux|MOOD=indic|TENSE=pres|PERSON=ps3|NUMBER=sg|VERB_PS=ps|VERB_POLARITY=af', 'pos': 'V', 'form': 'b'}]),
Span('kärmelt', [{'morphtag': 'POS=D', 'pos': 'D', 'form': ''}]),
Span('kulunud', [{'morphtag': 'POS=V|VERB_TYPE=main|VERB_FORM=partic|TENSE=past|VERB_PS=ps', 'pos': 'V', 'form': 'nud'}]),
Span('.', [{'morphtag': 'POS=Z|PUNCT_TYPE=Fst', 'pos': 'Z', 'form': ''}])])

In the output layer, attributes `'pos'`, `'form'` are based on Vabamorf's categories, and can be used to disambiguate the input `morph_analysis` layer; the attribute `'morphtag'` contains model-specific morphological features (Vabamorf's categories extended towards [UD](https://universaldependencies.org/guidelines.html) categories).

You can also use NeuralMorphDisambWebTagger directly as **a disambiguator of the input morph analysis layer**. 
For this, you need to change the output_layer of the tagger to morph analysis layer, and then use the tagger as a retagger:

In [3]:
# Initialize neural morph tagger as retagger
neural_morph_retagger = NeuralMorphDisambWebTagger(url='http://127.0.0.1:5000/estnltk/tagger/neural_morph_disamb', 
                                                   output_layer='morph_analysis')
# Create input text
text=Text('Kiirelt võetud pangalaen on kärmelt kulunud.').tag_layer('morph_analysis')
# Disambiguate morph layer
neural_morph_retagger.retag(text)
text['morph_analysis']

Layer(name='morph_analysis', attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech'), spans=SL[Span('Kiirelt', [{'normalized_text': 'Kiirelt', 'lemma': 'kiir', 'root': 'kiir', 'root_tokens': ['kiir'], 'ending': 'lt', 'clitic': '', 'form': 'sg abl', 'partofspeech': 'S'}]),
Span('võetud', [{'normalized_text': 'võetud', 'lemma': 'võetud', 'root': 'võetud', 'root_tokens': ['võetud'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'A'}]),
Span('pangalaen', [{'normalized_text': 'pangalaen', 'lemma': 'pangalaen', 'root': 'panga_laen', 'root_tokens': ['panga', 'laen'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'S'}]),
Span('on', [{'normalized_text': 'on', 'lemma': 'olema', 'root': 'ole', 'root_tokens': ['ole'], 'ending': '0', 'clitic': '', 'form': 'b', 'partofspeech': 'V'}]),
Span('kärmelt', [{'normalized_text': 'kärmelt', 'lemma': 'kärmelt', 'root': 'kärmelt', 'root_tokens': ['kärmelt'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'D'}]),
Span('kulunud', [{'normalized_text': 'kulunud', 'lemma': 'kuluma', 'root': 'kulu', 'root_tokens': ['kulu'], 'ending': 'nud', 'clitic': '', 'form': 'nud', 'partofspeech': 'V'}]),
Span('.', [{'normalized_text': '.', 'lemma': '.', 'root': '.', 'root_tokens': ['.'], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}])])

If you need to process large datasets, it is recommended to use neural morphological tagging locally.

## Running locally

*Note: if you want to run neural morphological tagging locally, you need to install [estnltk_neural](https://github.com/estnltk/estnltk/tree/main/estnltk_neural) package. Be aware that the implementation also requires an old `tensorflow` version (version < 2.0, such as 1.15.5), and is not compatible with the newest `tensorflow`.*

There are 4 morphological tagging models available, and for each of these models, there is a tagger class defined in `estnltk_neural.taggers`:

* `SoftmaxEmbTagSumTagger()`
* `SoftmaxEmbCatSumTagger()`
* `Seq2SeqEmbTagSumTagger()`
* `Seq2SeqEmbCatSumTagger()`

Note that models are not distributed with the estnltk\_neural package. You can download them in the following ways:
* If you create a new instance of tagger (`SoftmaxEmbTagSumTagger()`, `Seq2SeqEmbTagSumTagger()` etc) and the model has not been downloaded yet, you'll be prompted with a question asking for a permission to download the model;
* Alternatively, you can pre-download models manually via the download function:

```python
from estnltk import download
# download model for SoftmaxEmbCatSumTagger
download('softmaxembcatsumtagger')
# download model for Seq2SeqEmbTagSumTagger
download('seq2seqembtagsumtagger')
...
```

### `SoftmaxEmbTagSumTagger`

In [1]:
from estnltk import Text
from estnltk_neural.taggers import SoftmaxEmbTagSumTagger

text = Text("See on lause.")
text.tag_layer(['morph_analysis'])
        
tagger = SoftmaxEmbTagSumTagger('morph_softmax_emb_tag_sum')
tagger.tag(text)

This requires downloading resource 'neural_morph_softmax_emb_tag_sum_2019-08-23' (size: 354M). Proceed with downloading? [Y/n] y


Unpacked resource into subfolder 'neural_morph_disamb/softmax_emb_tag_sum_2019-08-23/' of the resources dir.
Loaded analyses: 341 from file C:\Programmid\Miniconda3\envs\py37_estnltk_neural\lib\site-packages\estnltk\estnltk_resources\neural_morph_disamb\softmax_emb_tag_sum_2019-08-23\output\data\analysis.txt



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which 

Text(text='See on lause.')

Now the text object has a layer named `morph_softmax_emb_tag_sum`, which contains three attributes for every word: `morphtag` (which is the original tag predicted by the neural model), `pos` and `form` (which are morptags converted into vabamorf format).

In [2]:
text['morph_softmax_emb_tag_sum']

Layer(name='morph_softmax_emb_tag_sum', attributes=('morphtag', 'pos', 'form'), spans=SL[Span('See', [{'morphtag': 'POS=P|NUMBER=sg|CASE=nom', 'pos': 'P', 'form': 'sg n'}]),
Span('on', [{'morphtag': 'POS=V|VERB_TYPE=main|MOOD=indic|TENSE=pres|PERSON=ps3|NUMBER=sg|VERB_PS=ps|VERB_POLARITY=af', 'pos': 'V', 'form': 'b'}]),
Span('lause', [{'morphtag': 'POS=S|NOUN_TYPE=com|NUMBER=sg|CASE=nom', 'pos': 'S', 'form': 'sg n'}]),
Span('.', [{'morphtag': 'POS=Z|PUNCT_TYPE=Fst', 'pos': 'Z', 'form': ''}])])

### `SoftmaxEmbCatSumTagger`

If you want to load a new tagger with a different neural model, you need to reset the previously loaded one with the `reset` method.

In [3]:
from estnltk_neural.taggers import SoftmaxEmbCatSumTagger

tagger.reset()
tagger = SoftmaxEmbCatSumTagger('morph_softmax_emb_cat_sum')
tagger.tag(text)
text['morph_softmax_emb_cat_sum']




INFO:model.py:79: Initializing tf session
INFO:model.py:92: Reloading the latest trained model...
INFO:saver.py:1284: Restoring parameters from C:\Programmid\Miniconda3\envs\py37_estnltk_neural\lib\site-packages\estnltk\estnltk_resources\neural_morph_disamb\softmax_emb_cat_sum_2019-08-23\output\results\model.weights


Layer(name='morph_softmax_emb_cat_sum', attributes=('morphtag', 'pos', 'form'), spans=SL[Span('See', [{'morphtag': 'POS=P|NUMBER=sg|CASE=nom', 'pos': 'P', 'form': 'sg n'}]),
Span('on', [{'morphtag': 'POS=V|VERB_TYPE=main|MOOD=indic|TENSE=pres|PERSON=ps3|NUMBER=sg|VERB_PS=ps|VERB_POLARITY=af', 'pos': 'V', 'form': 'b'}]),
Span('lause', [{'morphtag': 'POS=S|NOUN_TYPE=com|NUMBER=sg|CASE=nom', 'pos': 'S', 'form': 'sg n'}]),
Span('.', [{'morphtag': 'POS=Z|PUNCT_TYPE=Fst', 'pos': 'Z', 'form': ''}])])

### `Seq2SeqEmbTagSumTagger`

In [4]:
from estnltk_neural.taggers import Seq2SeqEmbTagSumTagger

tagger.reset()
tagger = Seq2SeqEmbTagSumTagger('morph_seq2seq_emb_tag_sum')
tagger.tag(text)
text['morph_seq2seq_emb_tag_sum']

Instructions for updating:
dim is deprecated, use axis instead
INFO:model.py:147: Initializing tf session
INFO:model.py:160: Reloading the latest trained model...
INFO:tf_logging.py:115: Restoring parameters from /home/paul/Projects/estnltk/estnltk/taggers/neural_morph/new_neural_morph/seq2seq_emb_tag_sum/output/results/model.weights


Layer(name='morph_seq2seq_emb_tag_sum', attributes=('morphtag', 'pos', 'form'), spans=SL[Span('See', [{'morphtag': 'POS=P|NUMBER=sg|CASE=nom', 'pos': 'P', 'form': 'sg n'}]),
Span('on', [{'morphtag': 'POS=V|VERB_TYPE=main|MOOD=indic|TENSE=pres|PERSON=ps3|NUMBER=sg|VERB_PS=ps|VERB_POLARITY=af', 'pos': 'V', 'form': 'b'}]),
Span('lause', [{'morphtag': 'POS=S|NOUN_TYPE=com|NUMBER=sg|CASE=nom', 'pos': 'S', 'form': 'sg n'}]),
Span('.', [{'morphtag': 'POS=Z|PUNCT_TYPE=Fst', 'pos': 'Z', 'form': ''}])])

### `Seq2SeqEmbCatSumTagger`

In [5]:
from estnltk_neural.taggers import Seq2SeqEmbCatSumTagger

tagger.reset()
tagger = Seq2SeqEmbCatSumTagger('morph_seq2seq_emb_cat_sum')
tagger.tag(text)
text['morph_seq2seq_emb_cat_sum']

INFO:model.py:147: Initializing tf session
INFO:model.py:160: Reloading the latest trained model...
INFO:tf_logging.py:115: Restoring parameters from /home/paul/Projects/estnltk/estnltk/taggers/neural_morph/new_neural_morph/seq2seq_emb_cat_sum/output/results/model.weights


Layer(name='morph_seq2seq_emb_cat_sum', attributes=('morphtag', 'pos', 'form'), spans=SL[Span('See', [{'morphtag': 'POS=P|NUMBER=sg|CASE=nom', 'pos': 'P', 'form': 'sg n'}]),
Span('on', [{'morphtag': 'POS=V|VERB_TYPE=main|MOOD=indic|TENSE=pres|PERSON=ps3|NUMBER=sg|VERB_PS=ps|VERB_POLARITY=af', 'pos': 'V', 'form': 'b'}]),
Span('lause', [{'morphtag': 'POS=S|NOUN_TYPE=com|NUMBER=sg|CASE=nom', 'pos': 'S', 'form': 'sg n'}]),
Span('.', [{'morphtag': 'POS=Z|PUNCT_TYPE=Fst', 'pos': 'Z', 'form': ''}])])